# Ranking documents using skip grams word embedding model for phrase queries

In [89]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import *
from nltk.stem.porter import *
import os
from Constants import *
import pickle
import gensim 
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [90]:
# ! Uncomment in first run 
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('stopwords')

## Setup
#### The corpus/documents are extracted from the pickle files.
#### Stemmer has been initialised to convert term to its root form (Example : received -> receive)

In [91]:
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))
root = Path(".")

my_path = root / "Pickled_files" / "Documents"
dbfile = open(my_path, 'rb')     
documents = pickle.load(dbfile)
dbfile.close()

## Model

Skip-gram model is a type of neural network architecture that is commonly used for word embedding. It is a form of unsupervised learning that aims to learn the distributional representation of words.The basic idea behind skip-gram model is to predict the context words surrounding a given target word, rather than predicting the target word from the context words. The context words are defined as the words that occur within a certain window of the target word.

The skip-gram model consists of an input layer, a hidden layer, and an output layer. The input layer represents the target word, and the output layer represents the context words. The hidden layer is used to transform the input word into a distributed representation, or embedding, which is used to predict the context words.During training, the skip-gram model is fed with a large corpus of text. For each target word in the corpus, a training example is created by randomly selecting one of its context words as the output, and setting the remaining context words as inputs to the model. The model is then trained to predict the output context word given the input context words.

The training process involves adjusting the weights of the model to minimize the difference between the predicted context word and the actual context word. The weights are adjusted using backpropagation algorithm, which updates the weights based on the difference between the predicted and actual output.After training, the skip-gram model generates a dense vector representation for each word in the vocabulary. The vector representation captures the semantic and syntactic information of the word, and can be used as input to other machine learning models for various natural language processing tasks such as text classification, information retrieval, and machine translation.

## Building the model

The following code initializes an empty list called data, which will be used to store the tokenized and preprocessed documents. Then, it loops over each document in the documents list and tokenizes each sentence into a list of words. Each word is then converted to lowercase and added to a temporary list called temp. The temp list is then appended to the data list, which now contains a list of tokenized and preprocessed documents.

Next, the Word2Vec model from the gensim library is used to train a word embedding model on the data list. The min_count parameter specifies the minimum frequency count of a word in the corpus for it to be included in the model. The vector_size parameter specifies the dimensionality of the word vectors. The window parameter specifies the maximum distance between the target word and the context word within a sentence. The sg parameter specifies the training algorithm to be used - 1 for skip-gram and 0 for CBOW. In this case, sg is set to 1 which corresponds to the skip-gram algorithm for training.

After training, the word embedding model skipgram_model contains dense vector representations for each word in the corpus. These vector representations can be used for various natural language processing tasks such as text classification, information retrieval, and machine translation.

In [102]:
data = list()
for i in documents: 
    temp = list()
    # tokenize the sentence into words 
    for j in word_tokenize(i[0]): 
        temp.append(j.lower()) 
    data.append(temp)

skipgram_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5,sg = 1)

## Generating word embeddings for documents

The following code defines two functions.

1.**preprocess(text)** takes a string of text as input, tokenizes it, removes stop words and filters out words with length less than two. It returns a list of preprocessed tokens.

2.**text_embedding(text, model)** takes a string of text and a pre-trained word embedding model as inputs. It preprocesses the text using the preprocess() function, obtains the word embeddings for each word in the preprocessed text from the pre-trained word embedding model, takes the average of the word embeddings to generate a document embedding, and returns the document embedding as a numpy array.

Finally, the **document_vecs** variable is assigned a list of document embeddings generated using the text_embedding() function and the pre-trained word embedding model (skipgram_model) for each document in the documents list.


In [105]:
def preprocess(text):
    tokens = []
    for word in word_tokenize(text, language='english'):
        if len(word) >= 2 and word not in stop_words:
            tokens.append(word)
    return tokens

# Define a function to generate text embeddings
def text_embedding(text, model):
    # Preprocess the text (tokenize, remove stop words, stem)
    preprocessed_text = preprocess(text)
    # Get the word embeddings for each word in the document
    word_embeddings = [model.wv[word] for word in preprocessed_text if word in model.wv.key_to_index]
    # Take the average of the word embeddings to generate a document embedding
    if len(word_embeddings) > 0:
        document_embedding = np.mean(word_embeddings, axis=0)
    else:
        document_embedding = np.zeros(model.vector_size)
    return document_embedding

document_vecs = [text_embedding(document[0], skipgram_model) for document in documents]

In [106]:
pd.DataFrame(document_vecs)

0         1         2         3         4         5         6   \
0     0.206675  0.054547 -0.014692 -0.003927 -0.026756 -0.420749  0.283177   
1     0.191518  0.253252 -0.036307 -0.008909 -0.018811 -0.477852  0.327341   
2     0.154748  0.157415 -0.026929 -0.196174 -0.063128 -0.440125  0.160929   
3     0.207999  0.176518  0.044923 -0.189760 -0.129191 -0.449643  0.183666   
4     0.174037  0.230450  0.001026 -0.161818 -0.086793 -0.433262  0.255819   
...        ...       ...       ...       ...       ...       ...       ...   
2373  0.160291  0.068060 -0.010850 -0.016243  0.042547 -0.301904  0.376517   
2374  0.116637  0.034341  0.036088  0.019712  0.009077 -0.402104  0.341002   
2375  0.122492  0.041628 -0.000005 -0.011336  0.059729 -0.387400  0.337439   
2376 -0.005365  0.023678  0.085230  0.090532  0.042343 -0.291634  0.256536   
2377  0.132930 -0.351242  0.092209  0.131784  0.075273 -0.101920  0.298432   

            7         8         9   ...        90        91        92  \
0     0.084087  0.136020  0.021542  ...  0.373114 -0.125953  0.171524   
1    -0.123260  0.100434  0.023203  ...  0.354657 -0.081819  0.128444   
2    -0.160880  0.195482  0.009681  ...  0.438138  0.054933  0.130219   
3    -0.187676  0.187501  0.037843  ...  0.538991 -0.123464  0.090113   
4    -0.219981  0.110463  0.040317  ...  0.356322 -0.076896  0.119087   
...        ...       ...       ...  ...       ...       ...       ...   
2373  0.118914  0.029595  0.051652  ...  0.393338 -0.097152  0.218863   
2374  0.067454  0.171476 -0.002749  ...  0.391155 -0.119265  0.182735   
2375  0.017183  0.117517 -0.011920  ...  0.391228 -0.072704  0.257140   
2376  0.096108  0.041206  0.034731  ...  0.183279 -0.113884  0.063614   
2377  0.471381  0.065388  0.046039  ...  0.400269 -0.302683  0.363408   

            93        94        95        96        97        98        99  
0    -0.145394  0.089098 -0.027650  0.143820 -0.260030  0.192953  0.160765  
1    -0.317003 -0.056611 -0.053263  0.373928 -0.222742  0.236504 -0.034385  
2    -0.266910  0.009393  0.017474  0.373477 -0.193283  0.284580  0.119897  
3    -0.247639  0.078290  0.094601  0.330541 -0.184547  0.155966  0.037984  
4    -0.304511  0.050431  0.111769  0.407474 -0.180155  0.247936  0.041905  
...        ...       ...       ...       ...       ...       ...       ...  
2373 -0.011125  0.152679  0.001062  0.132301 -0.178840  0.186988  0.025415  
2374 -0.147085  0.163016  0.110871  0.091892 -0.197560  0.121246  0.063576  
2375 -0.131345  0.232458  0.077982  0.127659 -0.233226  0.170853  0.066398  
2376 -0.062915  0.123683  0.187335  0.072833 -0.209256  0.011695 -0.006037  
2377  0.079542  0.476036  0.247744 -0.066857 -0.210577  0.064994 -0.014745  

[2378 rows x 100 columns]

## Similarity measure

Here cosine_similarity is used to compare the query vector and and the document vector.

In [108]:
# Define a function to calculate cosine similarity between two vectors
def cosine_similarity(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

## Saving document vectors for future use

In [110]:
my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'wb')
pickle.dump(document_vecs, dbfile) 
dbfile.close()

my_path = root / "Pickled_files" / "Document_vectors"
dbfile = open(my_path, 'rb')     
document_vecs = pickle.load(dbfile)
dbfile.close()

## Processing the query

This code calculates the similarity scores between a given query and a set of documents. It first generates a document vector for each document in the set using the text_embedding function. Then, it generates a query vector using the same function. The cosine similarity between the query vector and each document vector is calculated and stored in the similarity_scores list. Finally, the documents are ranked in descending order based on their similarity scores with the query, and the top five documents are printed.

In [111]:
# Example usage: compare a query with a set of documents
query = "Excluded driver"

query_vec = text_embedding(query,skipgram_model)
# Calculate similarity scores between the query and all documents
similarity_scores = [cosine_similarity(query_vec, document_vec) for document_vec in document_vecs]
# Rank documents based on similarity scores
ranked_documents = [document for _, document in sorted(zip(similarity_scores, documents), reverse=True)]
# Print the ranked documents
ranked_documents[0:5]

[('anyone injured while occupying an auto without a reasonable belief that he or she had the consent of the owner to do so.a household member , other than your spouse , while occupying or struck by an auto owned or regularly used by you or any household member unless a premium for this part is shown for that auto on the coverage selections page.you or your spouse , if a household member , while occupying or struck by an auto owned or regularly used by you or your spouse unless a premium for this part is shown for that auto on the coverage selections page . ',
  '.\\Docs\\Auto\\7thEditionPolicy.docx'),
 ('we will not pay damages to or for any household member who has a massachusetts auto policy of his or her own or who is covered by any massachusetts auto policy of another household member providing uninsured auto insurance with higher limits.anyone else while occupying your auto.we will not pay damages to or for anyone else who has a massachusetts auto policy of his or her own , or who